Introduction to NLP course (2017-2018).

Homework 3: Distributional semantic models.

Objectives:

1) Obtain co-occurrence vector representations with the followin properties:
- window size 1, pmi, svd (50)
- window size 3, no modifications
- window size 3, pmi, no svd
- window size 3, no pmi, svd (50)
- window size 3, pmi, svd (50)

2) Obtain word2vec embeddings with the following properties
- window size 1, 50 dimensions
- window size 1, 200 dimensions
- window size 3, 50 dimensions
- window size 3, 200 dimensions
- window size 5, 50 dimensions

3) Compare the performance of the 10 representations in 1 and 2 on the following tasks:
- similarity between "man" and "woman"
- the 5 most similar words to "car"
- for DISSECT representations , correlation with gold standard
- for Word2Vec, the similarity between "queen" and "king + woman - man"

In [1]:
# Import section
import nltk
from nltk.corpus import gutenberg
from nltk import FreqDist
from nltk.collocations import *
import re
from collections import Counter
import numpy as np
import operator
from scipy import spatial

# Dissect
from composes.semantic_space.space import Space
from composes.utils import io_utils
from composes.transformation.scaling.ppmi_weighting import PpmiWeighting
from composes.transformation.dim_reduction.svd import Svd
from composes.similarity.cos import CosSimilarity
from composes.utils import scoring_utils

# Gensim
import gensim 

SyntaxError: invalid syntax (space.py, line 206)

In [ ]:

## Load the corpus
corpus = gutenberg.words()


def getSpace(windowSize):
    ## generate the raw co-occurrence count within a window of 1
    cooc = BigramCollocationFinder.from_words(corpus,window_size=windowSize + 1).ngram_fd.items()

    ## convert the list of collocates in a dictionary

    # Initialize the dict
    cooc_dict = {}

    # Loop through the list
    for pair,freq in cooc:
        # Check and initialie the variables
        word1,word2 = pair
        # Check if entries for the words exist
        # If not, create them
        if word1 not in cooc_dict:
            cooc_dict[word1]={}

        if word2 not in cooc_dict:
            cooc_dict[word2]={}

        # Check if entries for the particular combination exists
        # If not, initialize them
        if word2 not in cooc_dict[word1]:
            cooc_dict[word1][word2]=0
        if word1 not in cooc_dict[word2]:
            cooc_dict[word2][word1]=0
        # Update the dict variables
        cooc_dict[word1][word2]+=freq
        cooc_dict[word2][word1]+=freq

    ## Generate the row, col and data variables for the DISSECT

    # Initialize the variables
    rows = []
    cols = []
    data = []

    # Loop through the dictionary
    for word_1 in cooc_dict:
        # Add an entry to the rows variable
        # there should be no duplications, but we check anyway
        if word_1 not in rows:
            rows.append(word_1)
        # Loop through the entries in the dict
        for word_2 in cooc_dict[word_1]:
            # Add an entry in the cols, if it's not already added
            if word_2 not in cols:
                cols.append(word_2)
            # Add the value to the data
            data.append(word_1 + " " + word_2 + " " + str(cooc_dict[word_1][word_2]))

    ## Output the row,col,data to files

    # Define the base name
    fname = "gutenberg_surface_3"

    # Generate tuples of fname data for the files
    out = []
    out.append((fname + ".rows",rows))
    out.append((fname + ".cols",cols))
    out.append((fname + ".sm",data))

    # Loop through the out var
    for (filename,content) in out:
        # Open the file
        with open(filename,"w") as out_file:
            # Loop through the rows variable
            for entry in content:
                # Remove non unicode chars
                entry = entry.encode('utf8', 'replace')
                # Write the entry
                out_file.write(entry)
                # Add newline
                out_file.write("\n")

    # Path to the folder where the data files are
    my_path = ""

    # Loading the matrix from the three different files
    my_space = Space.build(data = my_path + "gutenberg_surface_3.sm",
                           rows = my_path + "gutenberg_surface_3.rows",
                           cols = my_path + "gutenberg_surface_3.cols",
                           format = "sm")
    
    return my_space

# START TASK 1-1

space_ws1 = getSpace(1)

# Transforming the semantic space using PPMI
my_ppmi_space = space_ws1.apply(PpmiWeighting())

ws1_pmi_svd50_Task_1_1  = my_ppmi_space.apply(Svd(50))
# END TASK 1-1

# START TASK 1-2to5
space_ws3 = getSpace(3)
# Transforming the semantic space using PPMI
ppmi_space = space_ws3.apply(PpmiWeighting())

ws3_Task_1_2 = space_ws3
ws3_pmi_Task_1_3 = ppmi_space
ws3_svd50_Task_1_4 = space_ws3.apply(Svd(50))
ws3_pmi_svd50_Task_1_5 = ppmi_space.apply(Svd(50))

# END TASK 1-2to5
 
    
# START TASK 3    
    
# Comparing similarity between "man" and "woman"

print("Calculating similarity between man and woman")
print("window size 1, pmi, svd (50)")
print("ws1_pmi_svd50_Task_1_1", ws1_pmi_svd50_Task_1_1.get_sim("man", "woman", CosSimilarity()))

print("window size 3")
print("ws3_Task_1_2", ws3_Task_1_2.get_sim("man", "woman", CosSimilarity()))

print("window size 3, pmi")
print("ws3_pmi_Task_1_3", ws3_pmi_Task_1_3.get_sim("man", "woman", CosSimilarity()))

print("window size 3, svd (50)")
print("ws3_svd50_Task_1_4", ws3_svd50_Task_1_4.get_sim("man", "woman", CosSimilarity()))

print("window size 3, pmi, svd (50)")
print("ws3_pmi_svd50_Task_1_5", ws3_pmi_svd50_Task_1_5.get_sim("man", "woman", CosSimilarity()))


# Comparing the 5 most similar words to "car"

print("Obtaining the 5 most similar words to 'car'")
print("window size 1, pmi, svd (50)")
print("ws1_pmi_svd50_Task_1_1", ws1_pmi_svd50_Task_1_1.get_neighbours("car", 5, CosSimilarity()))

print("window size 3")
print("ws3_Task_1_2", ws3_Task_1_2.get_neighbours("car", 5, CosSimilarity()))

print("window size 3, pmi")
print("ws3_pmi_Task_1_3", ws3_pmi_Task_1_3.get_neighbours("car", 5, CosSimilarity()))

print("window size 3, svd (50)")
print("ws3_svd50_Task_1_4", ws3_svd50_Task_1_4.get_neighbours("car", 5, CosSimilarity()))

print("window size 3, pmi, svd (50)")
print("ws3_pmi_svd50_Task_1_5", ws3_pmi_svd50_Task_1_5.get_neighbours("car", 5, CosSimilarity()))

# Comparing the similarity with "gold standard"
fname = my_path + "synonyms.txt"
# Load the pairs
word_pairs = io_utils.read_tuple_list(fname, fields=[0,1])
# Load the score
gold = io_utils.read_list(fname, field=2)
# Predict similarity

print("Comparing similarity with 'gold standard'")
print("window size 1, pmi, svd (50)")
predicted_ppmi_svd = [round(sim,2) for sim in ws1_pmi_svd50_Task_1_1.get_sims(word_pairs, CosSimilarity())]
print ("Pairs:",word_pairs)
print ("Gold scores",gold)
print ("\n PPMI and SVD matrix:")
print ("Predicted scores",predicted_ppmi_svd)
print ("Spearman correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "spearman"))
print ("Pearson correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "pearson"))

print("window size 3")
predicted_ppmi_svd = [round(sim,2) for sim in ws3_Task_1_2.get_sims(word_pairs, CosSimilarity())]
print ("Pairs:",word_pairs)
print ("Gold scores",gold)
print ("\n PPMI and SVD matrix:")
print ("Predicted scores",predicted_ppmi_svd)
print ("Spearman correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "spearman"))
print ("Pearson correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "pearson"))

print("window size 3, pmi")
predicted_ppmi_svd = [round(sim,2) for sim in ws3_pmi_Task_1_3.get_sims(word_pairs, CosSimilarity())]
print ("Pairs:",word_pairs)
print ("Gold scores",gold)
print ("\n PPMI and SVD matrix:")
print ("Predicted scores",predicted_ppmi_svd)
print ("Spearman correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "spearman"))
print ("Pearson correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "pearson"))

print("window size 3, svd (50)")
predicted_ppmi_svd = [round(sim,2) for sim in ws3_svd50_Task_1_4.get_sims(word_pairs, CosSimilarity())]
print ("Pairs:",word_pairs)
print ("Gold scores",gold)
print ("\n PPMI and SVD matrix:")
print ("Predicted scores",predicted_ppmi_svd)
print ("Spearman correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "spearman"))
print ("Pearson correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "pearson"))

print("window size 3, pmi, svd (50)")
predicted_ppmi_svd = [round(sim,2) for sim in ws3_pmi_svd50_Task_1_5.get_sims(word_pairs, CosSimilarity())]
print ("Pairs:",word_pairs)
print ("Gold scores",gold)
print ("\n PPMI and SVD matrix:")
print ("Predicted scores",predicted_ppmi_svd)
print ("Spearman correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "spearman"))
print ("Pearson correlation:",scoring_utils.score(gold, predicted_ppmi_svd, "pearson"))

